In [1]:
import os
import numpy as np
import ROOT
from ROOT import TFile, TCanvas, TPad, TLegend, TMath, THistPainter, gStyle
c = ROOT.TCanvas()

directories = ("p_no_recoils", "p_recoils", "p_vacuum")

root_files_dict = {}

for directory in directories:  
    sim = sorted(f for f in os.listdir(directory) if f.endswith(".root"))  # Only ROOT files  
    root_files = [ROOT.TFile.Open(os.path.join(directory, f)) for f in sim[:4]]  # Open only ROOT files  
    root_files_dict[directory] = [f for f in root_files if f and not f.IsZombie()]  # Remove failed files  

histograms = ["hPtTrack", "hPtJet_R04", "hPtJetDPhiSignal_R04","hPtJetDPhiSignal_R03", "hNEvent", "hNtrig", "hPtGammaRatioPtJetDPhi_R04"]
data = {hist: {key: [] for key in directories} for hist in histograms}

for key, files in root_files_dict.items():
    for file in files:
        for hist in histograms:
            hist_obj = file.Get(hist)
            if hist_obj:
                data[hist][key].append(hist_obj.Clone())  # Clone ensures independent objects


print(data["hPtJetDPhiSignal_R04"]["p_vacuum"][0])



Welcome to JupyROOT 6.26/04
Name: hPtJetDPhiSignal_R04 Title: jet pt vs dphi R=0.4 NbinsX: 150


In [2]:

hEvents_vac = []
hEvents_nr = []
hEvents_r = []

ranges = range(len(data["hPtTrack"]["p_vacuum"]))
for evt in ranges:
    evt_vac_hist = data["hNEvent"]["p_vacuum"][evt]
    evt_nr_hist = data["hNEvent"]["p_no_recoils"][evt]
    evt_r_hist = data["hNEvent"]["p_recoils"][evt]

    hEvents_vac.append(evt_vac_hist)
    hEvents_r.append(evt_r_hist)
    hEvents_nr.append(evt_nr_hist)

    

vac_Entries = []
r_Entries = []
nr_Entries = []


for i in range(len(hEvents_vac)):
    ve = hEvents_vac[i].GetEntries() 
    nre = hEvents_nr[i].GetEntries()  
    re = hEvents_r[i].GetEntries()  

    # Store the values in the respective lists
    vac_Entries.append(ve)
    r_Entries.append(re)
    nr_Entries.append(nre)   
    
print(r_Entries[0])



print(data["hNtrig"]["p_vacuum"][0])

tEvents_vac = []
tEvents_nr = []
tEvents_r = []

ranges = range(len(data["hNtrig"]["p_vacuum"]))
for evt in ranges:
    t_vac_hist = data["hNtrig"]["p_vacuum"][evt]
    t_nr_hist = data["hNtrig"]["p_no_recoils"][evt]
    t_r_hist = data["hNtrig"]["p_recoils"][evt]

    tEvents_vac.append(t_vac_hist)
    tEvents_r.append(t_r_hist)
    tEvents_nr.append(t_nr_hist)

    

vac_trig = []
r_trig = []
nr_trig = []


for i in range(len(hEvents_vac)):
    vt = tEvents_vac[i].GetBinContent(2)  
    nrt = tEvents_nr[i].GetBinContent(2)    
    rt = tEvents_r[i].GetBinContent(2)    

    # Store the values in the respective lists
    vac_trig.append(vt)
    r_trig.append(rt)
    nr_trig.append(nrt)   
    
print(r_trig[0])


%jsroot on


abc = ROOT.TCanvas("abc", "Canvas", 1200, 1600)
abc.Divide(1, 2)

# Scale the histograms directly
for i in range(len(data["hPtJet_R04"]["p_vacuum"])):
    data["hPtTrack"]["p_vacuum"][i].Scale(1/vac_Entries[i])
    data["hPtJet_R04"]["p_vacuum"][i].Scale(1/vac_Entries[i])
    data["hPtJetDPhiSignal_R04"]["p_vacuum"][i].Scale(1/vac_trig[i]) 

for i in range(len(data["hPtJet_R04"]["p_no_recoils"])):
    data["hPtTrack"]["p_no_recoils"][i].Scale(1/nr_Entries[i])
    data["hPtJet_R04"]["p_no_recoils"][i].Scale(1/nr_Entries[i])
    data["hPtTrack"]["p_no_recoils"][i].Divide(data["hPtTrack"]["p_vacuum"][i])
    data["hPtJet_R04"]["p_no_recoils"][i].Divide(data["hPtJet_R04"]["p_vacuum"][i])

for i in range(len(data["hPtJet_R04"]["p_recoils"])):
    data["hPtTrack"]["p_recoils"][i].Scale(1/r_Entries[i])
    data["hPtJet_R04"]["p_recoils"][i].Scale(1/r_Entries[i])
    data["hPtTrack"]["p_recoils"][i].Divide(data["hPtTrack"]["p_vacuum"][i])
    data["hPtJet_R04"]["p_recoils"][i].Divide(data["hPtJet_R04"]["p_vacuum"][i])

 
                 


2033111.0
Name: hNtrig Title: number of triggers NbinsX: 2
0.706308126449585


In [3]:
def make_f_gauss(name="f_gauss", param_1 = 3.14, param_2 = 0.1, param_3 = 0.005):
    f = ROOT.TF1(name, "[0] * TMath::Exp((x - TMath::Pi()) / [1]) + [3]", np.pi/2, np.pi)
    f.SetParameter(0, param_1)
    f.SetParameter(1, param_2)
    f.SetParameter(2, param_3)
    f.SetParName(0, "Peak")
    f.SetParName(1, "Sigma")
    f.SetParName(2, "p1")
    return f

In [4]:
#----------------Vacuum  Delta Phi ------------------

%jsroot on


trig_bin_vac_1 = data["hNtrig"]["p_vacuum"][0].GetBinContent(2)
trig_bin_vac_2 = data["hNtrig"]["p_vacuum"][1].GetBinContent(2)
trig_bin_vac_3 = data["hNtrig"]["p_vacuum"][2].GetBinContent(2)
trig_bin_vac_4 = data["hNtrig"]["p_vacuum"][3].GetBinContent(2)



#--------------------------------
bin_fin1 = data["hPtJetDPhiSignal_R04"]["p_vacuum"][0].GetXaxis()
first_bin1 = bin_fin1.FindBin(10+0.1)
last_bin1 = bin_fin1.FindBin(100-0.1)

f_gauss1 = make_f_gauss("f_gauss1")

vac_interval1 = data["hPtJetDPhiSignal_R04"]["p_vacuum"][0].ProjectionY("interval_1",first_bin1,last_bin1)
vac_interval1.SetTitle("pT in Vaccum")
vac_interval1.Rebin(4)
#vac_interval1.Scale(1.0 / trig_bin_vac_1,"width")
vac_interval1.Scale(1.0 / vac_interval1.Integral())
vac_interval1.Fit(f_gauss1, "R")
vac_interval1.SetLineColor(ROOT.kOrange+7)

fit_func1 = vac_interval1.GetFunction("f_gauss1")
fit_func1.SetLineColor(ROOT.kOrange+7)

print("Chi2 / NDF:", fit_func1.GetChisquare(), "/", fit_func1.GetNDF())
print("Chi2 / NDF:", fit_func1.GetChisquare()/fit_func1.GetNDF())
#------------------------------------------
bin_fin2 = data["hPtJetDPhiSignal_R04"]["p_vacuum"][1].GetXaxis()
first_bin2 = bin_fin2.FindBin(10+0.1)
last_bin2 = bin_fin2.FindBin(100-0.1)

f_gauss2 = make_f_gauss("f_gauss2")

vac_interval2 = data["hPtJetDPhiSignal_R04"]["p_vacuum"][1].ProjectionY("interval_2",first_bin2,last_bin2)
vac_interval2.SetTitle("pT in Vaccum")
vac_interval2.Rebin(4)
#vac_interval2.Scale(1.0 / trig_bin_vac_2,"width")
vac_interval2.Scale(2.0 / vac_interval2.Integral())
vac_interval2.Fit(f_gauss2, "R")
vac_interval2.SetLineColor(ROOT.kPink+5)

fit_func2 = vac_interval2.GetFunction("f_gauss2")
fit_func2.SetLineColor(ROOT.kPink+5) 

print("Chi2 / NDF:", fit_func2.GetChisquare(), "/", fit_func2.GetNDF())
print("Chi2 / NDF:", fit_func2.GetChisquare()/fit_func2.GetNDF())
#------------------------------------------
bin_fin3 = data["hPtJetDPhiSignal_R04"]["p_vacuum"][2].GetXaxis()
first_bin3 = bin_fin3.FindBin(10+0.1)
last_bin3 = bin_fin3.FindBin(100-0.1)

f_gauss3 = make_f_gauss("f_gauss3",2,0.2)

vac_interval3 = data["hPtJetDPhiSignal_R04"]["p_vacuum"][2].ProjectionY("interval_3",first_bin3,last_bin3)
vac_interval3.SetTitle("pT in Vaccum")
vac_interval3.Rebin(4)
#vac_interval3.Scale(1.0 / trig_bin_vac_3,"width")
vac_interval3.Scale(3.0 / vac_interval3.Integral())
vac_interval3.Fit(f_gauss3, "R")
vac_interval3.SetLineColor(ROOT.kViolet+5)

fit_func3 = vac_interval3.GetFunction("f_gauss3")
fit_func3.SetLineColor(ROOT.kViolet+5)
print("Chi2 / NDF:", fit_func3.GetChisquare(), "/", fit_func3.GetNDF())
print("Chi2 / NDF:", fit_func3.GetChisquare() / fit_func3.GetNDF())
#------------------------------------------------
bin_fin4 = data["hPtJetDPhiSignal_R04"]["p_vacuum"][3].GetXaxis()
first_bin4 = bin_fin4.FindBin(10+0.1)
last_bin4 = bin_fin4.FindBin(100-0.1)

f_gauss4 = make_f_gauss("f_gauss4")

vac_interval4 = data["hPtJetDPhiSignal_R04"]["p_vacuum"][3].ProjectionY("interval_4",first_bin4,last_bin4)
vac_interval4.SetTitle("pT in Vaccum")
vac_interval4.Rebin(4)
#vac_interval4.Scale(1.0 / trig_bin_vac_4,"width")
vac_interval4.Scale(4.0 / vac_interval4.Integral())
vac_interval4.Fit(f_gauss4, "R")
vac_interval4.SetLineColor(ROOT.kAzure+5)

fit_func4 = vac_interval4.GetFunction("f_gauss4")
fit_func4.SetLineColor(ROOT.kAzure+5) 
print("Chi2 / NDF:", fit_func4.GetChisquare(), "/", fit_func4.GetNDF())
print("Chi2 / NDF:", fit_func4.GetChisquare()/fit_func4.GetNDF())
#------------------------------------------------
ROOT.gStyle.SetOptStat(0)

abcd = ROOT.TCanvas("abcd", "Canvas", 600, 400)
#ROOT.gPad.SetLogy(1)
vac_interval1.SetStats(False)
vac_interval1.GetXaxis().SetRangeUser(1.8,3.141)
vac_interval1.GetYaxis().SetRangeUser(0.0,1.2)
vac_interval1.SetTitle(" ")
vac_interval1.Draw()
vac_interval1.GetYaxis().SetTitle("number of recoiling jets (arb. normalisation)")
vac_interval2.Draw("SAME")
vac_interval3.Draw("SAME")
vac_interval4.Draw("SAME")

legendP = TLegend(0.2,0.55,0.4,0.85)
legendP.SetBorderSize(0)
legendP.SetFillColor(0)
legtext1 = formatted_string = f"#splitline{{photon pT = 10-20 GeV}}{{#sigma = {fit_func1.GetParameter(1):.2f} ± {fit_func1.GetParError(1):.2f}}}"
legtext2 = formatted_string = f"#splitline{{photon pT = 20-30 GeV}}{{#sigma = {fit_func2.GetParameter(1):.2f} ± {fit_func2.GetParError(1):.2f}}}"
legtext3 = formatted_string = f"#splitline{{photon pT = 30-50 GeV}}{{#sigma = {fit_func3.GetParameter(1):.2f} ± {fit_func3.GetParError(1):.2f}}}"
legtext4 = formatted_string = f"#splitline{{photon pT = 50-70 GeV}}{{#sigma = {fit_func4.GetParameter(1):.2f} ± {fit_func4.GetParError(1):.2f}}}"
legendP.AddEntry(vac_interval1,legtext1, "l")
legendP.AddEntry(vac_interval2, legtext2, "l")
legendP.AddEntry(vac_interval3, legtext3, "l")
legendP.AddEntry(vac_interval4, legtext4, "l")
legendP.Draw()

abcd.Draw()
abcd.SaveAs("Photon_Graphs/Vac_dephi.png")




Chi2 / NDF: 61.984424577089065 / 9
Chi2 / NDF: 6.8871582863432295
Chi2 / NDF: 268.50821603783277 / 9
Chi2 / NDF: 29.83424622642586
Chi2 / NDF: 1056.6197482783994 / 9
Chi2 / NDF: 117.40219425315549
Chi2 / NDF: 35.08205555945141 / 9
Chi2 / NDF: 3.898006173272379
 FCN=61.9844 FROM MIGRAD    STATUS=CONVERGED     451 CALLS         452 TOTAL
                     EDM=8.36906e-08    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   0.0 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Peak         3.75025e-01   5.01457e-03  -3.73034e-05   6.98652e-02
   2  Sigma        2.96572e-01   4.43774e-03   6.36981e-05  -1.11843e-02
   3  p1           5.00000e-03   1.73206e+00  -0.00000e+00   0.00000e+00
   4  p3           2.53198e-03   4.22775e-04  -2.30327e-06  -1.40976e+00
 FCN=268.508 FROM MIGRAD    STATUS=CONVERGED     255 CALLS         256 TOTAL
                     EDM=1.49218e-07    STRATEGY= 1  ERR

Error in <TFormula::SetParName>: Parameter p2 is not defined.
Error in <TFormula::SetParName>: Parameter p2 is not defined.
Error in <TFormula::SetParName>: Parameter p2 is not defined.
Error in <TFormula::SetParName>: Parameter p2 is not defined.
Info in <TCanvas::Print>: png file Photon_Graphs/Vac_dephi.png has been created


In [7]:
#------------------- Recoils deltaphi --------------

%jsroot on


trig_bin_r_1 = data["hNtrig"]["p_recoils"][0].GetBinContent(2)
trig_bin_r_2 = data["hNtrig"]["p_recoils"][1].GetBinContent(2)
trig_bin_r_3 = data["hNtrig"]["p_recoils"][2].GetBinContent(2)
trig_bin_r_4 = data["hNtrig"]["p_recoils"][3].GetBinContent(2)

r_interval1 = data["hPtJetDPhiSignal_R04"]["p_recoils"][0].ProjectionY("rinterval_1")
no_r_interval1 = data["hPtJetDPhiSignal_R04"]["p_no_recoils"][0].ProjectionY("norinterval_1")

f_gauss1 = make_f_gauss("f_gauss1")#,param_2 = 0.5, param_3 = 0.1)
f_gauss2 = make_f_gauss("f_gauss2")#,param_2 = 4, param_3 = 0.1)
f_gauss3 = make_f_gauss("f_gauss3")#,param_2 = 5,param_3 = 0.1)
f_gauss4 = make_f_gauss("f_gauss4")#, param_2 = 10, param_3 = 0.1)
#--------------------------------
bin_fin1 = data["hPtJetDPhiSignal_R04"]["p_recoils"][0].GetXaxis()
first_bin1 = bin_fin1.FindBin(10+0.1)
last_bin1 = bin_fin1.FindBin(100-0.1)

r_interval1 = data["hPtJetDPhiSignal_R04"]["p_recoils"][0].ProjectionY("interval_1",first_bin1,last_bin1)
r_interval1.SetTitle("pT p_recoils")
r_interval1.Rebin(4)
#vac_interval1.Scale(1.0 / trig_bin_vac_1,"width")
#scale so that all are the same integral for now
r_interval1.Scale(1.0 / r_interval1.Integral())
r_interval1.Fit(f_gauss1, "R")
r_interval1.SetLineColor(ROOT.kOrange+7)

fit_func1 = r_interval1.GetFunction("f_gauss1")
fit_func1.SetLineColor(ROOT.kOrange+7) 
#------------------------------------------
bin_fin2 = data["hPtJetDPhiSignal_R04"]["p_recoils"][1].GetXaxis()
first_bin2 = bin_fin2.FindBin(10+0.1)
last_bin2 = bin_fin2.FindBin(100-0.1)

r_interval2 = data["hPtJetDPhiSignal_R04"]["p_recoils"][1].ProjectionY("interval_2",first_bin2,last_bin2)
r_interval2.SetTitle("pT p_recoils")
r_interval2.Rebin(4)
#vac_interv
#r_interval2.Scale(1.0 / trig_bin_r_2,"width")
r_interval2.Scale(2.0 / r_interval2.Integral())
r_interval2.Fit(f_gauss2, "R")
r_interval2.SetLineColor(ROOT.kPink+5)

fit_func2 = r_interval2.GetFunction("f_gauss2")
fit_func2.SetLineColor(ROOT.kPink+5) 
#------------------------------------------
bin_fin3 = data["hPtJetDPhiSignal_R04"]["p_recoils"][2].GetXaxis()
first_bin3 = bin_fin3.FindBin(10+0.1)
last_bin3 = bin_fin3.FindBin(100-0.1)

r_interval3 = data["hPtJetDPhiSignal_R04"]["p_recoils"][2].ProjectionY("interval_3",first_bin3,last_bin3)
r_interval3.SetTitle("pT p_recoils")
r_interval3.Rebin(4)
#vac_interval3.Scale(1.0 / trig_bin_vac_3,"width")
r_interval3.Scale(3.0 / r_interval3.Integral())
r_interval3.Fit(f_gauss3, "R")
r_interval3.SetLineColor(ROOT.kViolet+5)

fit_func3 = r_interval3.GetFunction("f_gauss3")
fit_func3.SetLineColor(ROOT.kViolet+5) 
#------------------------------------------------
bin_fin4 = data["hPtJetDPhiSignal_R04"]["p_recoils"][3].GetXaxis()
first_bin4 = bin_fin4.FindBin(10+0.1)
last_bin4 = bin_fin4.FindBin(100-0.1)

r_interval4 = data["hPtJetDPhiSignal_R04"]["p_recoils"][3].ProjectionY("interval_4",first_bin4,last_bin4)
r_interval4.SetTitle("pT p_recoils")
r_interval4.Rebin(4)
#vac_interval4.Scale(1.0 / trig_bin_vac_4,"width")
r_interval4.Scale(4.0 / r_interval4.Integral())
r_interval4.Fit(f_gauss4, "R")
r_interval4.SetLineColor(ROOT.kAzure+5)

fit_func4 = r_interval4.GetFunction("f_gauss4")
fit_func4.SetLineColor(ROOT.kAzure+5) 

print("Chi2 / NDF:", fit_func1.GetChisquare(), "/", fit_func1.GetNDF())
print("Chi2 / NDF:", fit_func2.GetChisquare(), "/", fit_func2.GetNDF())
print("Chi2 / NDF:", fit_func3.GetChisquare(), "/", fit_func3.GetNDF())
print("Chi2 / NDF:", fit_func4.GetChisquare(), "/", fit_func4.GetNDF())
print("-----------------------------------")
print("Chi2 / NDF:", fit_func1.GetChisquare()/fit_func1.GetNDF())
print("Chi2 / NDF:", fit_func2.GetChisquare()/fit_func2.GetNDF())
print("Chi2 / NDF:", fit_func3.GetChisquare() / fit_func3.GetNDF())
print("Chi2 / NDF:", fit_func4.GetChisquare()/fit_func4.GetNDF())
#------------------------------------------------
ROOT.gStyle.SetOptStat(0)

abcde = ROOT.TCanvas("abcde", "Canvas", 600, 400)
ROOT.gPad.SetLogy(1)
r_interval1.SetStats(False)
r_interval1.GetXaxis().SetRangeUser(1.8,3.141)
r_interval1.GetYaxis().SetRangeUser(0.0,1.2)
r_interval1.SetTitle(" ")
r_interval1.Draw()
r_interval1.GetYaxis().SetTitle("number of recoiling jets (arb. normalisation)")
r_interval2.Draw("SAME")
r_interval3.Draw("SAME")
r_interval4.Draw("SAME")

legendP = TLegend(0.2,0.55,0.4,0.85)
legendP.SetBorderSize(0)
legendP.SetFillColor(0)
legtext1 = formatted_string = f"#splitline{{photon pT = 10-20 GeV}}{{#sigma = {fit_func1.GetParameter(1):.2f} ± {fit_func1.GetParError(1):.2f}}}"
legtext2 = formatted_string = f"#splitline{{photon pT = = 20-30 GeV}}{{#sigma = {fit_func2.GetParameter(1):.2f} ± {fit_func2.GetParError(1):.2f}}}"
legtext3 = formatted_string = f"#splitline{{photon pT = = 30-50 GeV}}{{#sigma = {fit_func3.GetParameter(1):.2f} ± {fit_func3.GetParError(1):.2f}}}"
legtext4 = formatted_string = f"#splitline{{photon pT = = 50-70 GeV}}{{#sigma = {fit_func4.GetParameter(1):.2f} ± {fit_func4.GetParError(1):.2f}}}"
legendP.AddEntry(r_interval1,legtext1, "l")
legendP.AddEntry(r_interval2, legtext2, "l")
legendP.AddEntry(r_interval3, legtext3, "l")
legendP.AddEntry(r_interval4, legtext4, "l")
legendP.Draw()

abcde.Draw()
abcde.SaveAs("Photon_Graphs/R_dephi.png")


Chi2 / NDF: 18.371123882168806 / 9
Chi2 / NDF: 23.32736509201115 / 9
Chi2 / NDF: 108.01385976444182 / 9
Chi2 / NDF: 220.86310605869787 / 9
-----------------------------------
Chi2 / NDF: 2.0412359869076453
Chi2 / NDF: 2.5919294546679055
Chi2 / NDF: 12.001539973826869
Chi2 / NDF: 24.540345117633095
 FCN=18.3711 FROM MIGRAD    STATUS=CONVERGED     493 CALLS         494 TOTAL
                     EDM=4.22695e-08    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   0.0 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Peak         3.16672e-01   1.60409e-02  -5.45197e-05  -1.91751e-02
   2  Sigma        3.73097e-01   2.13975e-02   9.41915e-05  -4.72424e-02
   3  p1           5.00000e-03   1.88719e+00  -0.00000e+00   0.00000e+00
   4  p3          -3.79200e-03   1.03812e-03  -3.76370e-06  -3.90104e-01
 FCN=23.3274 FROM MIGRAD    STATUS=CONVERGED     257 CALLS         258 TOTAL
                  

Error in <TFormula::SetParName>: Parameter p2 is not defined.
Error in <TFormula::SetParName>: Parameter p2 is not defined.
Error in <TFormula::SetParName>: Parameter p2 is not defined.
Error in <TFormula::SetParName>: Parameter p2 is not defined.
Warning in <TCanvas::Constructor>: Deleting canvas with same name: abcde
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Info in <TCanvas::Print>: png file Photon_Graphs/R_dephi.png has been created


In [6]:
#---------------- No Recoils deta Phi -------------------

%jsroot on


trig_bin_nr_1 = data["hNtrig"]["p_no_recoils"][0].GetBinContent(2)
trig_bin_nr_2 = data["hNtrig"]["p_no_recoils"][1].GetBinContent(2)
trig_bin_nr_3 = data["hNtrig"]["p_no_recoils"][2].GetBinContent(2)
trig_bin_nr_4 = data["hNtrig"]["p_no_recoils"][3].GetBinContent(2)


f_gauss1 = make_f_gauss("f_gauss1")#,param_2 = 0.5, param_3 = 0.1)
f_gauss2 = make_f_gauss("f_gauss2")#,param_2 = 4, param_3 = 0.1)
f_gauss3 = make_f_gauss("f_gauss3")#,param_2 = 5,param_3 = 0.1)
f_gauss4 = make_f_gauss("f_gauss4")#, param_2 = 100, param_3 = 1)

#--------------------------------
bin_fin1 = data["hPtJetDPhiSignal_R04"]["p_no_recoils"][0].GetXaxis()
first_bin1 = bin_fin1.FindBin(5+0.1)
last_bin1 = bin_fin1.FindBin(100-0.1)

nr_interval1 = data["hPtJetDPhiSignal_R04"]["p_no_recoils"][0].ProjectionY("interval_1",first_bin1,last_bin1)
nr_interval1.SetTitle("pT p_recoils")
nr_interval1.Rebin(4)
#vac_interval1.Scale(1.0 / trig_bin_vac_1,"width")
#scale so that all are the same integral for now
nr_interval1.Scale(1.0 / nr_interval1.Integral())
nr_interval1.Fit(f_gauss1, "R")
nr_interval1.SetLineColor(ROOT.kOrange+7)

fit_func1 = nr_interval1.GetFunction("f_gauss1")
fit_func1.SetLineColor(ROOT.kOrange+7) 
#------------------------------------------
bin_fin2 = data["hPtJetDPhiSignal_R04"]["p_no_recoils"][1].GetXaxis()
first_bin2 = bin_fin2.FindBin(5+0.1)
last_bin2 = bin_fin2.FindBin(100-0.1)

nr_interval2 = data["hPtJetDPhiSignal_R04"]["p_no_recoils"][1].ProjectionY("interval_2",first_bin2,last_bin2)
nr_interval2.SetTitle("pT p_recoils")
nr_interval2.Rebin(4)
#vac_interv
#nr_interval2.Scale(1.0 / trig_bin_r_2,"width")
nr_interval2.Scale(2.0 / nr_interval2.Integral())
nr_interval2.Fit(f_gauss2, "R")
nr_interval2.SetLineColor(ROOT.kPink+5)

fit_func2 = nr_interval2.GetFunction("f_gauss2")
fit_func2.SetLineColor(ROOT.kPink+5) 
#------------------------------------------
bin_fin3 = data["hPtJetDPhiSignal_R04"]["p_no_recoils"][2].GetXaxis()
first_bin3 = bin_fin3.FindBin(5+0.1)
last_bin3 = bin_fin3.FindBin(100-0.1)

nr_interval3 = data["hPtJetDPhiSignal_R04"]["p_no_recoils"][2].ProjectionY("interval_3",first_bin3,last_bin3)
nr_interval3.SetTitle("pT p_recoils")
nr_interval3.Rebin(4)
#vac_interval3.Scale(1.0 / trig_bin_vac_3,"width")
nr_interval3.Scale(3.0 / nr_interval3.Integral())
nr_interval3.Fit(f_gauss3, "R")
nr_interval3.SetLineColor(ROOT.kViolet+5)

fit_func3 = nr_interval3.GetFunction("f_gauss3")
fit_func3.SetLineColor(ROOT.kViolet+5) 
#------------------------------------------------
bin_fin4 = data["hPtJetDPhiSignal_R04"]["p_no_recoils"][3].GetXaxis()
first_bin4 = bin_fin4.FindBin(5+0.1)
last_bin4 = bin_fin4.FindBin(100-0.1)

nr_interval4 = data["hPtJetDPhiSignal_R04"]["p_no_recoils"][3].ProjectionY("interval_4",first_bin4,last_bin4)
nr_interval4.SetTitle("pT p_recoils")
nr_interval4.Rebin(4)
#vac_interval4.Scale(1.0 / trig_bin_vac_4,"width")
nr_interval4.Scale(4.0 / nr_interval4.Integral())
nr_interval4.Fit(f_gauss4, "R")
nr_interval4.SetLineColor(ROOT.kAzure+5)

fit_func4 = nr_interval4.GetFunction("f_gauss4")
fit_func4.SetLineColor(ROOT.kAzure+5) 

print("Chi2 / NDF:", fit_func1.GetChisquare(), "/", fit_func1.GetNDF())
print("Chi2 / NDF:", fit_func2.GetChisquare(), "/", fit_func2.GetNDF())
print("Chi2 / NDF:", fit_func3.GetChisquare(), "/", fit_func3.GetNDF())
print("Chi2 / NDF:", fit_func4.GetChisquare(), "/", fit_func4.GetNDF())
print("-----------------------------------")
print("Chi2 / NDF:", fit_func1.GetChisquare()/fit_func1.GetNDF())
print("Chi2 / NDF:", fit_func2.GetChisquare()/fit_func2.GetNDF())
print("Chi2 / NDF:", fit_func3.GetChisquare() / fit_func3.GetNDF())
print("Chi2 / NDF:", fit_func4.GetChisquare()/fit_func4.GetNDF())

#------------------------------------------------
ROOT.gStyle.SetOptStat(0)

abcdef = ROOT.TCanvas("abcdef", "Canvas", 600, 400)
#ROOT.gPad.SetLogy(1)
nr_interval1.SetStats(False)
nr_interval1.GetXaxis().SetRangeUser(1.8,3.141)
nr_interval1.GetYaxis().SetRangeUser(0.0,1.2)
nr_interval1.SetTitle(" ")
nr_interval1.Draw()
nr_interval1.GetYaxis().SetTitle("number of recoiling jets (arb. normalisation)")
nr_interval2.Draw("SAME")
nr_interval3.Draw("SAME")
nr_interval4.Draw("SAME")

legendP = TLegend(0.2,0.55,0.4,0.85)
legendP.SetBorderSize(0)
legendP.SetFillColor(0)
legtext1 = formatted_string = f"#splitline{{photon pT = 10-20 GeV}}{{#sigma = {fit_func1.GetParameter(1):.2f} ± {fit_func1.GetParError(1):.2f}}}"
legtext2 = formatted_string = f"#splitline{{photon pT = 20-30 GeV}}{{#sigma = {fit_func2.GetParameter(1):.2f} ± {fit_func2.GetParError(1):.2f}}}"
legtext3 = formatted_string = f"#splitline{{photon pT = 30-50 GeV}}{{#sigma = {fit_func3.GetParameter(1):.2f} ± {fit_func3.GetParError(1):.2f}}}"
legtext4 = formatted_string = f"#splitline{{photon pT = 50-70 GeV}}{{#sigma = {fit_func4.GetParameter(1):.2f} ± {fit_func4.GetParError(1):.2f}}}"
legendP.AddEntry(nr_interval1,legtext1, "l")
legendP.AddEntry(nr_interval2, legtext2, "l")
legendP.AddEntry(nr_interval3, legtext3, "l")
legendP.AddEntry(nr_interval4, legtext4, "l")
legendP.Draw()

abcdef.Draw()
abcdef.SaveAs("Photon_Graphs/NR_dephi.png")



Chi2 / NDF: 62.00291478441215 / 9
Chi2 / NDF: 110.51252742989266 / 9
Chi2 / NDF: 545.012127120198 / 9
Chi2 / NDF: 1412.2091125053603 / 9
-----------------------------------
Chi2 / NDF: 6.889212753823572
Chi2 / NDF: 12.279169714432518
Chi2 / NDF: 60.55690301335533
Chi2 / NDF: 156.9121236117067
 FCN=62.0029 FROM MIGRAD    STATUS=CONVERGED     505 CALLS         506 TOTAL
                     EDM=3.82821e-07    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   0.0 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Peak         3.38008e-01   3.95827e-03   1.45613e-05   1.86912e-01
   2  Sigma        3.60204e-01   5.71191e-03   4.13510e-05   3.70694e-01
   3  p1           5.00000e-03   1.76834e+00  -0.00000e+00   0.00000e+00
   4  p3          -1.11862e-03   5.69739e-04  -4.50977e-06   4.00309e+00
 FCN=110.513 FROM MIGRAD    STATUS=CONVERGED     270 CALLS         271 TOTAL
                     ED

Error in <TFormula::SetParName>: Parameter p2 is not defined.
Error in <TFormula::SetParName>: Parameter p2 is not defined.
Error in <TFormula::SetParName>: Parameter p2 is not defined.
Error in <TFormula::SetParName>: Parameter p2 is not defined.
Info in <TCanvas::Print>: png file Photon_Graphs/NR_dephi.png has been created


In [87]:
#~~~~
bin_fin_z = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][3].GetZaxis()
first_bin_z = bin_fin_z.FindBin(1.4+0.1)
last_bin_z = bin_fin_z.FindBin(3.14159-0.1)

bin_fin_y = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][3].GetYaxis()
first_bin_y = bin_fin_y.FindBin(10+0.1)
last_bin_y = bin_fin_y.FindBin(200-0.1)

trig_bin_nr = data["hNtrig"]["p_no_recoils"][3].GetBinContent(2)

nr_interval = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][3].ProjectionX("interval_nr", iymin = first_bin_y, iymax = last_bin_y, izmin = first_bin_z, izmax = last_bin_z)
nr_interval.SetTitle("no recoils Jet pT Photon E ratio")
#nr_interval.Rebin(3)
#nr_interval.Scale(1.0 / trig_bin_nr)#,"width")
#scale so that all are the same integral for now
nr_interval.Scale(1.0 / nr_interval.Integral())
#nr_interval.Fit(f_gauss, "R")
nr_interval.SetLineColor(ROOT.kPink-3)

#~~~~~~~~~~~~~~~~~~~
bin_fin_vac_z = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][3].GetZaxis()
first_bin_vac_z = bin_fin_vac_z.FindBin(1.4+0.1)
last_bin_vac_z = bin_fin_vac_z.FindBin(3.14159-0.1)

bin_fin_vac_y = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][3].GetYaxis()
first_bin_vac_y = bin_fin_y.FindBin(10+0.1)
last_bin_vac_y = bin_fin_y.FindBin(200-0.1)

trig_bin_vac = data["hNtrig"]["p_vacuum"][3].GetBinContent(2)

vac_interval = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][3].ProjectionX("interval_vac", iymin = first_bin_vac_y, iymax = last_bin_vac_y, izmin = first_bin_vac_z, izmax = last_bin_vac_z)
vac_interval.SetTitle("p_vacuum Jet pT Photon E ratio")
#nr_interval.Rebin(3)
#vac_interval.Scale(3.0 / trig_bin_vac)#,"width")
#scale so that all are the same integral for now
vac_interval.Scale(1.0 / vac_interval.Integral())
#nr_interval.Fit(f_gauss, "R")
vac_interval.SetLineColor(ROOT.kBlue-3)

#~~~~~~~~~~~~~~~

bin_fin_r_z = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][3].GetZaxis()
first_bin_r_z = bin_fin_z.FindBin(1.4+0.1)
last_bin_r_z = bin_fin_z.FindBin(3.14159-0.1)

bin_fin_r_y = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][3].GetYaxis()
first_bin_r_y = bin_fin_y.FindBin(10+0.1)
last_bin_r_y = bin_fin_y.FindBin(200-0.1)

trig_bin_r = data["hNtrig"]["p_recoils"][3].GetBinContent(2)

r_interval = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][3].ProjectionX("interval_r", iymin = first_bin_r_y, iymax = last_bin_r_y, izmin = first_bin_r_z, izmax = last_bin_r_z)
r_interval.SetTitle("p_recoils Jet pT Photon E ratio")
#nr_interval.Rebin(3)
#r_interval.Scale(3.0 / trig_bin_r)#,"width")
#scale so that all are the same integral for now
r_interval.Scale(1.0 / r_interval.Integral())
r_interval.Fit(f_gauss, "R")
r_interval.SetLineColor(ROOT.kViolet-3)

#~~~~~~~~~~~~~~
ROOT.gStyle.SetOptStat(0)

abcdefg = ROOT.TCanvas("abcdefg", "Canvas", 600, 400)
#ROOT.gPad.SetLogy(1)
nr_interval.SetStats(False)
nr_interval.GetXaxis().SetRangeUser(0,2)
nr_interval.GetYaxis().SetRangeUser(0.0,0.5)
#nr_interval.SetTitle("photon+jet #Delta#phi distribution, jet pT = 10-200 GeV")
nr_interval.Draw()
vac_interval.Draw("SAME")
r_interval.Draw("SAME")
#nr_interval.GetYaxis().SetTitle("number of recoiling jets (arb. normalisation)")


legendP = TLegend(0.2,0.55,0.4,0.85)
legendP.SetBorderSize(0)
legendP.SetFillColor(0)
legtext1 =  "No p_recoils 30-50"
legtext2 = "p_recoils 30-50"
legtext3 = "p_vacuum 30-50"
#legtext4 = formatted_string = f"#splitline{{photon pT = 50-70 GeV}}{{#sigma = {fit_func4.GetParameter(1):.2f} ± {fit_func4.GetParError(1):.2f}}}"
legendP.AddEntry(nr_interval,legtext1, "l")
legendP.AddEntry(r_interval, legtext2, "l")
legendP.AddEntry(vac_interval, legtext3, "l")
#legendP.AddEntry(nr_interval4, legtext4, "l")
legendP.Draw()

abcdefg.Draw()
abcdefg.SaveAs("Ratio_NR.png")

 FCN=59.4257 FROM MIGRAD    STATUS=CONVERGED     124 CALLS         125 TOTAL
                     EDM=2.34111e-09    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   1.6 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Peak        -1.44874e-05   1.23665e-05  -6.27272e-08   1.39971e+01
   2  Sigma        7.50557e-02   2.06300e-07  -6.05289e-10  -3.85701e+01
   3  p1           2.04466e-06   7.76096e-07   1.43905e-10   2.42722e+02


Info in <TCanvas::Print>: png file Ratio_NR.png has been created


In [148]:
%jsroot on

###~~~~~~~~~~~~~~~~~~~24MAR~~~~~~~~~~~~~~~`

#~~~~
bin_fin_z = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][3].GetZaxis()
first_bin_z = bin_fin_z.FindBin(1.4+0.1)
last_bin_z = bin_fin_z.FindBin(3.14159-0.1)

bin_fin_y = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][3].GetYaxis()
first_bin_y = bin_fin_y.FindBin(10+0.1)
last_bin_y = bin_fin_y.FindBin(100-0.1)

trig_bin_nr = data["hNtrig"]["p_no_recoils"][3].GetBinContent(2)

nr_interval = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][3].ProjectionX("interval_nr", iymin = first_bin_y, iymax = last_bin_y, izmin = first_bin_z, izmax = last_bin_z)
nr_interval.SetTitle("Jet pT and Photon E ratio")

nr_interval.Scale(4.0 / nr_interval.Integral())
#nr_interval.Fit(f_gauss, "R")
nr_interval.SetLineColor(ROOT.kPink+7)

#~~~~~~~~~~~~~~~~~~~
bin_fin_vac_z = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][3].GetZaxis()
first_bin_vac_z = bin_fin_vac_z.FindBin(1.4+0.1)
last_bin_vac_z = bin_fin_vac_z.FindBin(3.14159-0.1)

bin_fin_vac_y = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][3].GetYaxis()
first_bin_vac_y = bin_fin_y.FindBin(10+0.1)
last_bin_vac_y = bin_fin_y.FindBin(100-0.1)

trig_bin_vac = data["hNtrig"]["p_vacuum"][3].GetBinContent(2)

vac_interval = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][3].ProjectionX("interval_vac", iymin = first_bin_vac_y, iymax = last_bin_vac_y, izmin = first_bin_vac_z, izmax = last_bin_vac_z)
vac_interval.SetTitle("p_vacuum Jet pT Photon E ratio")

vac_interval.Scale(4.0 / vac_interval.Integral())
#nr_interval.Fit(f_gauss, "R")
vac_interval.SetLineColor(ROOT.kAzure+7)

#~~~~~~~~~~~~~~~

bin_fin_r_z = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][3].GetZaxis()
first_bin_r_z = bin_fin_z.FindBin(1.4+0.1)
last_bin_r_z = bin_fin_z.FindBin(3.14159-0.1)

bin_fin_r_y = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][3].GetYaxis()
first_bin_r_y = bin_fin_y.FindBin(10+0.1)
last_bin_r_y = bin_fin_y.FindBin(100-0.1)

trig_bin_r = data["hNtrig"]["p_recoils"][3].GetBinContent(2)

r_interval = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][3].ProjectionX("interval_r", iymin = first_bin_r_y, iymax = last_bin_r_y, izmin = first_bin_r_z, izmax = last_bin_r_z)
r_interval.SetTitle("p_recoils Jet pT Photon E ratio")

r_interval.Scale(4.0 / r_interval.Integral())

#nr_interval.Fit(f_gauss, "R")
r_interval.SetLineColor(ROOT.kViolet+7)

#~~~~~~~~~~~~~~
ROOT.gStyle.SetOptStat(0)

abcdefg = ROOT.TCanvas("abcdefg", "Canvas", 800, 600)
#ROOT.gPad.SetLogy(1)
nr_interval.SetStats(False)
nr_interval.GetXaxis().SetRangeUser(0,2)
#nr_interval.GetYaxis().SetRangeUser(0.0,1)
#nr_interval.SetTitle("photon+jet #Delta#phi distribution, jet pT = 10-200 GeV")
nr_interval.Draw()
vac_interval.Draw("SAME")
r_interval.Draw("SAME")
#nr_interval.GetYaxis().SetTitle("number of recoiling jets (arb. normalisation)")


legendP = TLegend(0.55, 0.7, 0.85, 0.85)
legendP.SetBorderSize(0)
legendP.SetFillColor(0)
legtext1 =  "No recoil effects, Photon: 50-70 GeV"
legtext2 = "Recoil effects, Photon: 50-70 GeV"
legtext3 = "p_vacuum, Photon: 50-70 GeV"
#legtext4 = formatted_string = f"#splitline{{photon pT = 50-70 GeV}}{{#sigma = {fit_func4.GetParameter(1):.2f} ± {fit_func4.GetParError(1):.2f}}}"
legendP.AddEntry(nr_interval,legtext1, "l")
legendP.AddEntry(r_interval, legtext2, "l")
legendP.AddEntry(vac_interval, legtext3, "l")
#legendP.AddEntry(nr_interval4, legtext4, "l")
legendP.Draw()

abcdefg.Draw()
abcdefg.SaveAs("Ratio_NR.png")
fit_fun = nr_interval.GetFunction("f_gauss")

Warning in <TCanvas::Constructor>: Deleting canvas with same name: abcdefg
Info in <TCanvas::Print>: png file Ratio_NR.png has been created


In [123]:


%jsroot on

###~~~~~~~~~~~~~~~~~~~ 30-50 Version ~~~~~~~~~~~~~~~`

#~~~~
bin_fin_z = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][3].GetZaxis()
first_bin_z = bin_fin_z.FindBin(1.4+0.1)
last_bin_z = bin_fin_z.FindBin(3.14159-0.1)

bin_fin_y = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][3].GetYaxis()
first_bin_y = bin_fin_y.FindBin(10+0.1)
last_bin_y = bin_fin_y.FindBin(200-0.1)

trig_bin_nr = data["hNtrig"]["p_no_recoils"][3].GetBinContent(2)

nr_interval = data["hPtGammaRatioPtJetDPhi_R04"]["p_no_recoils"][3].ProjectionX("interval_nr", iymin = first_bin_y, iymax = last_bin_y, izmin = first_bin_z, izmax = last_bin_z)
nr_interval.SetTitle("Jet pT and Photon E ratio")

#nr_interval.Scale(3.0 / nr_interval.Integral())
nr_interval.Scale(1/nr_trig[3])

nr_interval.SetLineColor(ROOT.kPink+7)

#--------------------------------------------------------------------------------------------------------------
bin_fin_vac_z = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][3].GetZaxis()
first_bin_vac_z = bin_fin_vac_z.FindBin(1.4+0.1)
last_bin_vac_z = bin_fin_vac_z.FindBin(3.14159-0.1)

bin_fin_vac_y = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][3].GetYaxis()
first_bin_vac_y = bin_fin_y.FindBin(10+0.1)
last_bin_vac_y = bin_fin_y.FindBin(200-0.1)

trig_bin_vac = data["hNtrig"]["p_vacuum"][3].GetBinContent(2)

vac_interval = data["hPtGammaRatioPtJetDPhi_R04"]["p_vacuum"][3].ProjectionX("interval_vac", iymin = first_bin_vac_y, iymax = last_bin_vac_y, izmin = first_bin_vac_z, izmax = last_bin_vac_z)
vac_interval.SetTitle("p_vacuum Jet pT Photon E ratio")
vac_interval.Scale(1/vac_trig[3])
#vac_interval.Scale(3.0 / vac_interval.Integral())
vac_interval.Scale(1/vac_trig[3])
vac_interval.SetLineColor(ROOT.kAzure+7)

#--------------------------------------------------------------------------------------------------------------

bin_fin_r_z = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][3].GetZaxis()
first_bin_r_z = bin_fin_z.FindBin(1.4+0.1)
last_bin_r_z = bin_fin_z.FindBin(3.14159-0.1)

bin_fin_r_y = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][3].GetYaxis()
first_bin_r_y = bin_fin_y.FindBin(10+0.1)
last_bin_r_y = bin_fin_y.FindBin(200-0.1)

trig_bin_r = data["hNtrig"]["p_recoils"][3].GetBinContent(2)

r_interval = data["hPtGammaRatioPtJetDPhi_R04"]["p_recoils"][3].ProjectionX("interval_r", iymin = first_bin_r_y, iymax = last_bin_r_y, izmin = first_bin_r_z, izmax = last_bin_r_z)
r_interval.SetTitle("p_recoils Jet pT Photon E ratio")

#r_interval.Scale(3.0 / r_interval.Integral())
r_interval.Scale(1/r_trig[3])

r_interval.SetLineColor(ROOT.kViolet+7)

#--------------------------------------------------------------------------------------------------------------
ROOT.gStyle.SetOptStat(0)

abcdefg = ROOT.TCanvas("abcdefg", "Canvas", 800, 600)
#ROOT.gPad.SetLogy(1)
nr_interval.SetStats(False)
nr_interval.GetXaxis().SetRangeUser(0,2)
nr_interval.GetYaxis().SetRangeUser(0.0,1)

nr_interval.Draw()
vac_interval.Draw("SAME")
r_interval.Draw("SAME")



legendP = TLegend(0.55, 0.7, 0.85, 0.85)
legendP.SetBorderSize(0)
legendP.SetFillColor(0)

legtext1 =  "No recoil effects, Photon: 30-50 GeV"
legtext2 = "Recoil effects, Photon: 30-50 GeV"
legtext3 = "p_vacuum, Photon: 30-50 GeV"

legendP.AddEntry(nr_interval,legtext1, "l")
legendP.AddEntry(r_interval, legtext2, "l")
legendP.AddEntry(vac_interval, legtext3, "l")

legendP.Draw()

abcdefg.Draw()
abcdefg.SaveAs("Ratio_NR.png")
fit_fun = nr_interval.GetFunction("f_gauss")

Warning in <TCanvas::Constructor>: Deleting canvas with same name: abcdefg
Info in <TCanvas::Print>: png file Ratio_NR.png has been created


In [153]:
#vac_trig = Vaccum.Get("hNtrig")
#r_trig =  p_recoils.Get("hNtrig")
#nor_trig = nop_recoils.Get("hNtrig")

#trig_bin_vac = vac_trig.GetBinContent(2)
#trig_bin_r = r_trig.GetBinContent(2)
#trig_bin_nor = nor_trig.GetBinContent(2)
r_interval1 = data["hPtJetDPhiSignal_R04"]["p_recoils"][0].ProjectionY("rinterval_1", first_bin, last_bin)
nor_interval1 = data["hPtJetDPhiSignal_R04"]["p_no_recoils"][0].ProjectionY("norinterval_1", first_bin,last_bin)
vac_interval1 = data["hPtJetDPhiSignal_R04"]["p_vacuum"][0].ProjectionY("interval_1", first_bin, last_bin)

#normalising the graphs

r_interval1.Scale(1.0 / trig_bin_r_1)
nor_interval1.Scale(1.0 / trig_bin_nr_1)
vac_interval1.Scale(1.0 / trig_bin_vac_1)

nor_interval1.SetLineColor(ROOT.kOrange+10) 
vac_interval1.SetLineColor(ROOT.kViolet+10) 
r_interval1.SetLineColor(ROOT.kTeal+9) 
#Plotting ^-^

c = ROOT.TCanvas("c1", "Canvas", 1200, 800)
 

nor_interval1.Draw()
r_interval1.Draw("SAME")
vac_interval1.Draw("SAME")

legendP = TLegend()
legendP.SetBorderSize(0)
legendP.SetFillColor(0)
legendP.AddEntry(nor_interval1, "No p_recoils", "l")
legendP.AddEntry(vac_interval1, "Vaccum", "l")
legendP.AddEntry(r_interval1, "p_recoils", "l")
legendP.Draw()

c.Draw()
c.SaveAs("ALL_PROJECTIONS.png")


NameError: name 'first_bin' is not defined

In [43]:
############### Junk Drawer March edition




#for directory in directories:  
#    sim = sorted(f for f in os.listdir(directory) if f.endswith(".root"))  # Only keep ROOT files  
#    root_files = [ROOT.TFile.Open(os.path.join(directory, f)) for f in sim[:3]]  # Open only ROOT files  

#    root_files_dict[directory] = [f for f in root_files if f and not f.IsZombie()]  # Remove failed files  

#events = range(len(root_files_dict["p_vacuum"]))  # Adjust to available files  

#for no in events:
#    vac_pttrack_event = root_files_dict["p_vacuum"][no].Get("hPtTrack")
#    r_pttrack_event = root_files_dict["p_recoils"][no].Get("hPtTrack")
#    nr_pttrack_event = root_files_dict["p_no_recoils"][no].Get("hPtTrack")

#    vac_jetpt_event = root_files_dict["p_vacuum"][no].Get("hPtJet_R03")
#    r_jetpt_event = root_files_dict["p_recoils"][no].Get("hPtJet_R03")
#    nr_jetpt_event = root_files_dict["p_no_recoils"][no].Get("hPtJet_R03")

#    vac_phi_signal = root_files_dict["p_vacuum"][no].Get("hPtJetDPhiSignal_R03")
#    r_phi_signal = root_files_dict["p_recoils"][no].Get("hPtJetDPhiSignal_R03")
#    nr_phi_signal = root_files_dict["p_no_recoils"][no].Get("hPtJetDPhiSignal_R03")
    
#vac_pttrack_event_1, vac_pttrack_event_2, vac_pttrack_event_3, vac_pttrack_event_NR_3 = vac_pttrack_event
#r_pttrack_event_1, r_pttrack_event_2, r_pttrack_event_3, r_pttrack_event_3 = r_pttrack_event
#nr_pttrack_event_1, nr_pttrack_event_2, nr_pttrack_event_3, nr_pttrack_event_3 = nr_pttrack_event

#vac_jetpt_event_1, vac_jetpt_event_2, vac_jetpt_event_3, vac_jetpt_event_3 = vac_jetpt_event
#r_jetpt_event_1, r_jetpt_event_2, r_jetpt_event_3, r_jetpt_event_3 = r_jetpt_event
#nr_jetpt_event_1, nr_jetpt_event_2, nr_jetpt_event_3, nr_jetpt_event_3 = nr_jetpt_event

#vac_phi_signal_1, vac_phi_signal_2, vac_phi_signal_3, vac_phi_signal_3 = vac_phi_signal
#r_phi_signal_1, r_phi_signal_2, r_phi_signal_3, r_phi_signal_3 = r_phi_signal
#nr_phi_signal_1, nr_phi_signal_2, nr_phi_signal_3, nr_phi_signal_3 = nr_phi_signal

In [69]:
#~~~~~~~~~~~~~~~~~~ CODE JUNK DRAWER ~~~~~~~~~~~~~~~~


# Get the content of the first bin
#nEvents = vac_event.GetBinContent(1)
#print("Content of the first bin:", nEvents)

# Draw the histogram

#vac_event.Draw()


#pad = TPad("pad", "Logarithmic y-axis pad", 0, 0, 1, 1)
#pad.SetLogy(1)  # Set y-axis to logarithmic scale
#pad.Draw()
#pad.cd() 


#total_weight = vac_event.GetSumOfWeights()
#entries = vac_event.GetEntries()

#print("Total Weight (Sum of Bin Contents):", total_weight)
#print("Total Entries (Count of Events):", entries)


In [ ]:

%jsroot on
c = ROOT.TCanvas("c", "Canvas", 1600, 600)
c.Divide(2, 1)
c.cd(1)  
ROOT.gPad.SetLogy(1)

data["hPtTrack"]["p_vacuum"][0].SetLineColor(ROOT.kRed)
data["hPtTrack"]["p_vacuum"][1].SetLineColor(ROOT.kBlue)
data["hPtTrack"]["p_vacuum"][3].SetLineColor(ROOT.kOrange)
data["hPtTrack"]["p_vacuum"][3].SetLineColor(ROOT.kGreen)
data["hPtTrack"]["p_vacuum"][0].Draw()
data["hPtTrack"]["p_vacuum"][1].Draw("SAME")
data["hPtTrack"]["p_vacuum"][3].Draw("SAME")
data["hPtTrack"]["p_vacuum"][3].Draw("SAME")
data["hPtTrack"]["p_vacuum"][0].SetTitle("Track pT for p_vacuum at 10 - 20 pT")

c.cd(2)  
ROOT.gPad.SetLogy(1)
data["hPtJet_R03"]["p_vacuum"][0].SetLineColor(ROOT.kRed)
data["hPtJet_R03"]["p_vacuum"][1].SetLineColor(ROOT.kBlue)
data["hPtJet_R03"]["p_vacuum"][3].SetLineColor(ROOT.kOrange)
data["hPtJet_R03"]["p_vacuum"][3].SetLineColor(ROOT.kGreen)
data["hPtJet_R03"]["p_vacuum"][0].Draw()
data["hPtJet_R03"]["p_vacuum"][1].Draw("SAME")
data["hPtJet_R03"]["p_vacuum"][3].Draw("SAME")
data["hPtJet_R03"]["p_vacuum"][3].Draw("SAME")
data["hPtJet_R03"]["p_vacuum"][0].SetTitle("Jet pT for p_vacuum at 10 - 20 pT")

c.SaveAs("Photon_Graphs/p_jet_partcile_pt.png")

c.Draw()